# Erste Datenabfragen Aufgaben Team Janis & Johann
## Import der Packages

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from src.database import read_sql
from src.load_icd10_mapping import load_icd_mapping
import os
import numpy as np

## Wie viele Tage nach Aufnahme wird die Hauptdiagnose eingetragen?

In [ ]:
df = read_sql("""
SELECT "CreationTime_day_of_visit" AS creationtime_day_of_visit
FROM student_data.vwd_diagnosen
WHERE "isPrimary" = TRUE;""")

df.creationtime_day_of_visit.describe()

Im Median wird die Hauptdiagnose am zweiten Tag nach Aufnahme eingetragen. Es gibt jedoch auch Patient:innen, bei denen die Diagnose bereits 29 Tage vor Aufnahme eingetragen wird. Wir vermuten, dass es sich hierbei um elektive Patient:innen handelt.

In [ ]:
df.plot(kind="hist",
       bins=range(int(df['creationtime_day_of_visit'].min()), int(df['creationtime_day_of_visit'].max()) + 1, 1),
        rwidth=0.8,
        color='#86bf91',
        alpha=0.7,
       title="Hauptdiagnose Eintrag nach x Tagen")

### Wie viel Tage nach Aufnahme werden die Nebendiagnosen eingetragen?

In [ ]:
df = read_sql("""
SELECT d."CreationTime_day_of_visit" AS creationtime_day_of_visit
FROM student_data.vwd_diagnosen AS d JOIN student_data.vwd_faelle AS F ON d.visit_oid = f.visit_oid
WHERE d."isPrimary" = FALSE
AND f.startingvisitoid = f.visit_oid;""")

df.creationtime_day_of_visit.describe()

Die Nebendiagnosen werden im Median ebenfalls nach 2 Tagen eingetragen.

In [ ]:
# Define the bin edges based on the desired width
bin_width = 1
bin_edges = range(int(df['creationtime_day_of_visit'].min()), int(df['creationtime_day_of_visit'].max()) + bin_width, bin_width)

df.plot(kind="hist",
       bins=bin_edges,
        rwidth=0.8,
        color='#86bf91',
        alpha=0.7,
       title="Nebendiagnosen Eintrag in Tagen")

## Wie ist die Geschlechterverteilung bei Lungenkrebs?

In [ ]:
df = read_sql("""
SELECT p.sex, COUNT(DISTINCT d.patient_oid) AS patient_count
FROM student_data.vwd_diagnosen d
JOIN student_data.vwd_faelle p ON d.patient_oid = p.patient_oid
WHERE d."Code" LIKE 'C34%%'
GROUP BY p.sex;""")

df["percentage"] = round(df.patient_count / sum(df.patient_count) * 100., 2)
df

Der Anteil männlicher Patienten an allen Fällen mit Lungenkrebs liegt bei fast 2/3.

## Wann werden Fälle aufgenommen?
Wir nutzen hier die verstrichene Zeit seit Aufnahme bis zur Eintragung der Diagnose (**CreationTime_minutes_since_start**), um vom Zeitpunkt der Eintragung (**CreationTime_hour_of_day**) zurück zu rechnen. Somit erhalten wir die Aufnahmeuhrzeit.

In [ ]:
df = read_sql("""
SELECT MOD("CreationTime_hour_of_day" + 24 - MOD(CAST("CreationTime_minutes_since_start" / 60.0 AS BIGINT), 24), 24) AS start_hour
FROM student_data.vwd_diagnosen WHERE "isPrimary" = True;""")

df.head()

In [ ]:
plt.hist(df.start_hour,
        bins=24,
        align="left",
        alpha=0.8)
plt.xticks(range(24))
plt.title("Aufnahmeuhrzeit")

In [ ]:
df.start_hour.describe()

## Wie viele Schlaganfallpatient:innen sind 40 Jahre alt oder jünger?

In [ ]:
df = read_sql("""
SELECT CAST(sum(CASE WHEN birth_decade >= 1980 THEN 1 ELSE 0 END) AS FLOAT) / COUNT(*) * 100 AS percentage_infarct_under_40
FROM student_data.vwd_diagnosen AS d JOIN student_data.vwd_faelle AS f ON d.visit_oid = f.visit_oid
WHERE ("Code" ILIKE 'I64%' OR "Code" ILIKE 'I63%')
AND f.startingvisitoid = f.visit_oid;""")

print(f"{round(df.iloc[0, 0], 2)}% aller Schlaganfallpatient:innen sind 1980 oder später geboren.")

## Wie viel Zeit vergeht im Durchschnitt, bis es die ersten Laborwerte gibt?
Verifizieren dass nur eine Hauptdiagnose pro Fall:

In [ ]:
read_sql("""
with primary_counts as (
	SELECT count(*)
	from student_data.vwd_diagnosen as d join student_data.vwd_faelle as f
	on D.visit_oid = F.visit_oid where d."isPrimary" = true group by F.startingvisitoid limit 10
) select * from primary_counts where "count" != 1""")

Es gibt nur eine Hauptdiagnose pro Fall.

Wir suche nun pro Fall die ersten Labor/Bildgebungsergebnisse. Ignoriere alle Werte, die vor der Aufnahme entstanden sind. Wir beschränken die Suche auf 100.000 Fälle, da die Abfrage sonst zu lang dauert.

In [ ]:
df = read_sql("""
WITH t AS (select min(observationdatetime_minutes_since_start) AS time_to_first_results_in_minutes
FROM student_data.vwd_investigationresults AS i 
JOIN student_data.vwd_faelle AS f
ON i.patientvisit_oid = f.visit_oid
GROUP BY F.startingvisitoid LIMIT 1000) 
SELECT * FROM t WHERE time_to_first_results_in_minutes > 0 """)

df.head()

In [ ]:
df["time_to_first_results_in_hours"] = round(df["time_to_first_results_in_minutes"] / 60., 2)

In [ ]:
def plot_loghist(x, bins):
    logbins = np.logspace(np.log10(np.min(x)),
                          np.log10(np.max(x)),
                          bins+1)
    plt.hist(x, bins=logbins)
    plt.xscale('log')

In [ ]:
plot_loghist(df.time_to_first_results_in_hours, 10)
plt.title("Stunden seit Aufnahme")

In [ ]:
df.time_to_first_results_in_hours.describe()

## Was sind die 10 häufigsten Erkrankungen, mit denen man im KEVB liegt?

Wir berücksichtigen hier nur die Hauptdiagnosen.

In [ ]:
top_diagnoses = read_sql("""
SELECT
  CASE
    WHEN POSITION('.' IN d."Code") > 0 THEN
      SUBSTRING(d."Code" FROM 1 FOR POSITION('.' IN d."Code") - 1)
    ELSE
      d."Code"
  END AS code_group,
  COUNT(d."Code") AS code_count
FROM
  student_data.vwd_diagnosen AS d
JOIN
  student_data.vwd_faelle AS f ON d.visit_oid = f.visit_oid
WHERE
  d."isPrimary" = true
  AND F.startingvisitoid = f.visit_oid
GROUP BY
  code_group
ORDER BY
  code_count DESC
LIMIT 10;
""")
icd_df = load_icd_mapping()
top_diagnoses = pd.merge(top_diagnoses,
               icd_df[["key_number_no_dot", "three_digit_title"]].drop_duplicates(),
               left_on="code_group",
               right_on="key_number_no_dot",
               how="left").drop("key_number_no_dot", axis=1)
top_diagnoses